In [1]:
# 1. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 2. ติดตั้งไลบรารีที่จำเป็น
!pip install transformers datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!unzip /content/drive/MyDrive/pineapple-ripeness-classification.zip

In [5]:
# 3. โหลดข้อมูล
from datasets import load_dataset

dataset = load_dataset('imagefolder', data_dir='/content/train/train', split='train')

Resolving data files:   0%|          | 0/2138 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
# 4. พรีโพรเซสข้อมูล
from transformers import AutoImageProcessor

model_name = "google/vit-base-patch16-224-in21k"
image_processor = AutoImageProcessor.from_pretrained(model_name)

def transform(example_batch):
    # รับภาพและ label จาก dataset
    images = [x.convert("RGB") for x in example_batch['image']]
    labels = example_batch['label']
    # พรีโพรเซสภาพ
    inputs = image_processor(images, return_tensors='pt')
    # เพิ่ม labels ลงใน inputs
    inputs['labels'] = labels
    return inputs

prepared_ds = dataset.with_transform(transform)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


In [7]:
# 5. โหลดโมเดล ViT ที่ผ่านการฝึกมาก่อน
from transformers import AutoModelForImageClassification

labels = dataset.features['label'].names
model = AutoModelForImageClassification.from_pretrained(
    model_name,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
)


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
# 6. ฝึกโมเดล (Fine-tuning)
from transformers import TrainingArguments, Trainer
import numpy as np

def compute_metrics(p):
    return {"accuracy": (np.argmax(p.predictions, axis=1) == p.label_ids).mean()}

training_args = TrainingArguments(
  output_dir="./vit-base-beans",
  per_device_train_batch_size=8,
  evaluation_strategy="steps",
  num_train_epochs=4,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=2e-4,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='none',
)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds,
    eval_dataset=prepared_ds,
    tokenizer=image_processor,
)

trainer.train()

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-9-7b79ea19d04c>:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss,Accuracy
100,0.393800,0.409274,0.854069
200,0.345700,0.438927,0.851263
300,0.225200,0.289694,0.903648
400,0.203100,0.239523,0.916745
500,0.332400,0.214891,0.933115
600,0.136800,0.157889,0.941534
700,0.208100,0.119106,0.962114
800,0.181100,0.084413,0.975210
900,0.129100,0.064158,0.983162
1000,0.062900,0.051605,0.988775


TrainOutput(global_step=1072, training_loss=0.25552231087279853, metrics={'train_runtime': 673.643, 'train_samples_per_second': 12.695, 'train_steps_per_second': 1.591, 'total_flos': 6.627352538780467e+17, 'train_loss': 0.25552231087279853, 'epoch': 4.0})

In [10]:
# 7. ประเมินผลและบันทึกโมเดล
trainer.evaluate()
trainer.save_model("/content/drive/MyDrive/vit_model")

In [11]:
import os
import pandas as pd
from PIL import Image
import torch
from transformers import AutoImageProcessor, AutoModelForImageClassification

In [13]:
# ========================
# 1. Define Paths
# ========================
MODEL_PATH = "/content/drive/MyDrive/vit_model"
TEST_FOLDER = "/content/test/test"
CSV_OUTPUT = "/content/submission_ppap.csv"

# ========================
# 2. Load Model & Processor
# ========================
model = AutoModelForImageClassification.from_pretrained(MODEL_PATH)
image_processor = AutoImageProcessor.from_pretrained(MODEL_PATH)

# Ensure model is in evaluation mode
model.eval()

# ========================
# 3. Load and Predict
# ========================
results = []
file_names = []

for img_name in os.listdir(TEST_FOLDER):
    img_path = os.path.join(TEST_FOLDER, img_name)
    if not img_name.lower().endswith(('png', 'jpg', 'jpeg')):
        continue  # Skip non-image files

    image = Image.open(img_path).convert("RGB")
    inputs = image_processor(image, return_tensors="pt")

    with torch.no_grad():
        outputs = model(**inputs)
        predicted_class = outputs.logits.argmax(-1).item()

    file_names.append(img_name)
    results.append(predicted_class)

# ========================
# 4. Save Predictions to CSV
# ========================
df = pd.DataFrame({"FileName": file_names, "class": results})
df.to_csv(CSV_OUTPUT, index=False)
print(f"Predictions saved to {CSV_OUTPUT}")

Predictions saved to /content/submission_ppap.csv
